In [7]:
import numpy as np
import pandas as pd
import yfinance as yf
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import matplotlib.pyplot as plt
import seaborn as sns
import requests


In [ ]:
# Step 1: Fetch S&P 500 Data (Last 2 Years)
session = requests.Session()
sp500 = yf.download('^GSPC', period='2y', interval='1d', session=session)
sp500['Return'] = sp500['Close'].pct_change()
sp500.dropna(inplace=True)

In [ ]:
# Step 2: Prepare Data for LSTM
lags = 6  # Using last 6 days to predict next day's return
X, y = [], []
for i in range(len(sp500) - lookback):
    window = sp500['Return'].iloc[i:i+lookback].values.flatten()  # Ensure it's a 1D array
    if len(window) == lookback:  # Ensure full 6-day window
        X.append(window)
        y.append(sp500['Return'].iloc[i+lookback])

X, y = np.array(X), np.array(y)  # Convert to NumPy arrays

print("X shape before reshaping:", X.shape)  # Should be (num_samples, 6)

# Now reshape for LSTM (LSTM expects 3D: [samples, time steps, features])
X = X.reshape(X.shape[0], X.shape[1], 1)

print("Final X shape:", X.shape) 

In [ ]:
# Step 3: Define LSTM Model
model = Sequential([
    LSTM(50, return_sequences=False, input_shape=(lags, 1)),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')

In [ ]:
# Step 4: Train Model
model.fit(X, y, epochs=10, batch_size=16, verbose=1)

In [ ]:
# Step 5: Compute Vanilla Gradients
with tf.GradientTape() as tape:
    inputs = tf.convert_to_tensor(X, dtype=tf.float32)
    tape.watch(inputs)
    preds = model(inputs)
grads = tape.gradient(preds, inputs)

In [ ]:
# Step 6: Convert to Numpy and Visualize
grads_np = grads.numpy().squeeze()

def plot_gradients(grads_np):
    plt.figure(figsize=(10, 6))
    for i in range(grads_np.shape[1]):  # Loop through each lag
        plt.plot(grads_np[:, i], label=f'Lag {i+1}')
    plt.xlabel("Sample Index")
    plt.ylabel("Gradient Value")
    plt.title("LSTM Gradients for Each Lag")
    plt.legend()
    plt.show()

plot_gradients(grads_np)

In [ ]:
# Display some results
print("Sample Gradients for first few inputs:")
print(grads_np[:5])